In [1]:
from pathlib import Path
import librosa
import numpy as np
import pandas as pd
import parselmouth
import os

# Path to your RAVDESS dataset folder (adjust as needed)
data_folder = Path(r"C:/Users/jbkee/OneDrive/Desktop/Jupyter Projects/RAVDESS/audio_speech_actors_01-24")

# RAVDESS emotion codes mapping
emotion_map = {
    "01": "neutral",
    "02": "calm",
    "03": "happy",
    "04": "sad",
    "05": "angry",
    "06": "fearful",
    "07": "disgust",
    "08": "surprised"
}

def safe_formant_values(formant_obj, formant_index, duration):
    times = np.arange(0, duration, 0.01)
    values = [formant_obj.get_value_at_time(formant_index, t) for t in times]
    return [v for v in values if v is not None and not np.isnan(v) and v > 0]

def extract_features(audio_path):
    y, sr = librosa.load(audio_path, sr=16000)
    y, _ = librosa.effects.trim(y, top_db=20)
    y = y / np.max(np.abs(y)) if np.max(np.abs(y)) > 0 else y

    # MFCC
    mfcc = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=13)
    mfcc_mean = np.mean(mfcc, axis=1)
    mfcc_std = np.std(mfcc, axis=1)
    mfcc_delta = librosa.feature.delta(mfcc)
    mfcc_delta2 = librosa.feature.delta(mfcc, order=2)

    # Mean of each delta coefficient
    mfcc_delta_mean = [np.mean(mfcc_delta[i]) for i in range(13)]
    mfcc_delta2_mean = [np.mean(mfcc_delta2[i]) for i in range(13)]
    
    # Pitch
    pitch = librosa.yin(y, fmin=50, fmax=500, sr=sr)
    pitch_mean = np.mean(pitch)
    pitch_std = np.std(pitch)

    

    # Spectral Centroid
    centroid = librosa.feature.spectral_centroid(y=y, sr=sr)
    centroid_mean = np.mean(centroid)
    centroid_std = np.std(centroid)
    
    # Spectral Bandwidth
    bandwidth = librosa.feature.spectral_bandwidth(y=y, sr=sr)
    bandwidth_mean = np.mean(bandwidth)
    bandwidth_std = np.std(bandwidth)
    
    # Spectral Rolloff
    rolloff = librosa.feature.spectral_rolloff(y=y, sr=sr)
    rolloff_mean = np.mean(rolloff)
    rolloff_std = np.std(rolloff)
    
    # Spectral Contrast
    contrast = librosa.feature.spectral_contrast(y=y, sr=sr)
    contrast_mean = np.mean(contrast, axis=1)
    contrast_std = np.std(contrast, axis=1)
    
    # Spectral Flatness
    flatness = librosa.feature.spectral_flatness(y=y)
    flatness_mean = np.mean(flatness)
    flatness_std = np.std(flatness)



    # RMS Energy
    rms = librosa.feature.rms(y=y)
    rms_mean = np.mean(rms)
    rms_std = np.std(rms)
    
    # Zero Crossing Rate
    zcr = librosa.feature.zero_crossing_rate(y)
    zcr_mean = np.mean(zcr)
    zcr_std = np.std(zcr)




    
    

    try:
        snd = parselmouth.Sound(str(audio_path))
        formant = snd.to_formant_burg()
        f1 = safe_formant_values(formant, 1, snd.duration)
        f2 = safe_formant_values(formant, 2, snd.duration)
        f3 = safe_formant_values(formant, 3, snd.duration)

        f1_mean = np.mean(f1) if f1 else 0
        f2_mean = np.mean(f2) if f2 else 0
        f3_mean = np.mean(f3) if f3 else 0
    except:
        f1_mean = f2_mean = f3_mean = 0  # fallback if extraction fails

    return {
        **{f"mfcc{i+1}_mean": mfcc[i].mean() for i in range(13)},
        **{f"mfcc_delta{i+1}_mean": mfcc_delta_mean[i] for i in range(13)},
        **{f"mfcc_delta2_{i+1}_mean": mfcc_delta2_mean[i] for i in range(13)},
        
        "pitch_mean": pitch_mean,
        "pitch_std": pitch_std,
        
        "f1_mean": f1_mean,
        "f2_mean": f2_mean,
        "f3_mean": f3_mean,

        "centroid_mean": centroid_mean,
        "centroid_std": centroid_std,
        "bandwidth_mean": bandwidth_mean,
        "bandwidth_std": bandwidth_std,
        "rolloff_mean": rolloff_mean,
        "rolloff_std": rolloff_std,
        "flatness_mean": flatness_mean,
        "flatness_std": flatness_std,
        **{f"contrast{i+1}_mean": contrast_mean[i] for i in range(contrast.shape[0])},
        **{f"contrast{i+1}_std": contrast_std[i] for i in range(contrast.shape[0])},

        "rms_mean": rms_mean,
        "rms_std": rms_std,
        "zcr_mean": zcr_mean,
        "zcr_std": zcr_std, 
    }

# Gather all features
feature_rows = []

for file in data_folder.glob("**/*.wav"):
    try:
        emotion_code = file.stem.split("-")[2]  # e.g., '03' for happy
        label = emotion_map.get(emotion_code, "unknown")
        
        feats = extract_features(file)
        feats["filename"] = file.name
        feats["label"] = label
        feature_rows.append(feats)
        
        print(f"Processed {file.name}")
    except Exception as e:
        print(f"Failed on {file.name}: {e}")

# Convert to DataFrame and save
df = pd.DataFrame(feature_rows)
df.to_csv("features_ravdess_03.csv", index=False)
print("✅ Features saved to features_ravdess.csv")

Processed 03-01-01-01-01-01-01.wav
Processed 03-01-01-01-01-02-01.wav
Processed 03-01-01-01-02-01-01.wav
Processed 03-01-01-01-02-02-01.wav
Processed 03-01-02-01-01-01-01.wav
Processed 03-01-02-01-01-02-01.wav
Processed 03-01-02-01-02-01-01.wav
Processed 03-01-02-01-02-02-01.wav
Processed 03-01-02-02-01-01-01.wav
Processed 03-01-02-02-01-02-01.wav
Processed 03-01-02-02-02-01-01.wav
Processed 03-01-02-02-02-02-01.wav
Processed 03-01-03-01-01-01-01.wav
Processed 03-01-03-01-01-02-01.wav
Processed 03-01-03-01-02-01-01.wav
Processed 03-01-03-01-02-02-01.wav
Processed 03-01-03-02-01-01-01.wav
Processed 03-01-03-02-01-02-01.wav
Processed 03-01-03-02-02-01-01.wav
Processed 03-01-03-02-02-02-01.wav
Processed 03-01-04-01-01-01-01.wav
Processed 03-01-04-01-01-02-01.wav
Processed 03-01-04-01-02-01-01.wav
Processed 03-01-04-01-02-02-01.wav
Processed 03-01-04-02-01-01-01.wav
Processed 03-01-04-02-01-02-01.wav
Processed 03-01-04-02-02-01-01.wav
Processed 03-01-04-02-02-02-01.wav
Processed 03-01-05-0